In [1]:
import pandas as pd
import numpy as np 
import os
import csv
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from tensorflow.keras import layers, models
import sklearn.metrics as metrics


In [2]:
path=''
all_files=glob.glob(path + "/*.csv")



In [3]:
li=[]
for filename in all_files:
    df=pd.read_csv(filename,header=0)
    min_val = df[' Value'].min()
    max_val = df[' Value'].max()

    df['NORMALIZED'] = ((df[' Value'] - min_val) / (max_val - min_val)) * 2 - 1
    li.append(df)

In [4]:
df=pd.concat(li,ignore_index=True)

In [6]:
df.drop(['Timestamp'], axis=1 ,inplace=True)
df.drop ([' Value'], axis=1 ,inplace=True)


In [7]:
print (df[' label'].unique())
print (df[' label'].value_counts())

[' noise' ' quake']
 label
 noise    9155870
 quake     112544
Name: count, dtype: int64


In [8]:
df.rename(columns = {' label':'Class'},inplace = True)
df

,Class,NORMALIZED
0,noise,0.022648
1,noise,0.016282
2,noise,0.031324
3,noise,0.037746
4,noise,0.022085
...,...,...
9268409,noise,-0.039234
9268410,noise,-0.039289
9268411,noise,-0.039343
9268412,noise,-0.039270


In [9]:
le=LabelEncoder()
df.Class= le.fit_transform(df.Class)

In [10]:
X=df.drop(['Class'],axis=1)
y=df['Class']

In [12]:
import pandas as pd
from collections import Counter
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Summarize class distribution
counter = Counter(y)
print(counter)

# Define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3)

# Undersample the dataset
X_res, y_res = undersample.fit_resample(X, y)

# Summarize the new class distribution
counter_resampled = Counter(y_res)
print(counter_resampled)




Counter({0: 9155870, 1: 112544})
Counter({0: 112544, 1: 112544})


In [13]:
print (X_res.shape)
print(y_res.shape)

(225088, 1)
(225088,)


In [14]:
count_greater_than_1 = (df['NORMALIZED'] > 1).sum()
print (count_greater_than_1)


0


In [17]:
X_train,X_test,y_train,y_test=train_test_split (X_res,y_res, test_size=0.2, random_state=42)


In [23]:
model_quake = models.Sequential(name='model_quake')
model_quake.add(layers.LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=True))
model_quake.add(layers.Flatten())
model_quake.add(layers.Dense(32, activation='relu'))
model_quake.add(layers.Dense(16, activation='relu'))
model_quake.add(layers.Dense(1, activation='sigmoid'))


In [24]:
model_quake.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [25]:
history=model_quake.fit(X_train, y_train, batch_size=64, epochs=12)

y_pred = model_quake.predict(X_test)

# Convert the predictions to binary format
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate evaluation metrics on the test set
precision = metrics.precision_score(y_test, y_pred_binary, zero_division=0)
recall = metrics.recall_score(y_test, y_pred_binary)
f1_score = metrics.f1_score(y_test, y_pred_binary)
auc_score = metrics.roc_auc_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("AUC Score:", auc_score)

Epoch 1/12
2814/2814 [==============================] - 6s 2ms/step - loss: 0.6090 - accuracy: 0.6960
Epoch 2/12
2814/2814 [==============================] - 4s 2ms/step - loss: 0.5491 - accuracy: 0.7543
Epoch 3/12
2814/2814 [==============================] - 4s 1ms/step - loss: 0.5377 - accuracy: 0.7579
Epoch 4/12
2814/2814 [==============================] - 4s 1ms/step - loss: 0.5297 - accuracy: 0.7590
Epoch 5/12
2814/2814 [==============================] - 5s 2ms/step - loss: 0.5235 - accuracy: 0.7587
Epoch 6/12
2814/2814 [==============================] - 5s 2ms/step - loss: 0.5184 - accuracy: 0.7590
Epoch 7/12
2814/2814 [==============================] - 4s 2ms/step - loss: 0.5143 - accuracy: 0.7588
Epoch 8/12
2814/2814 [==============================] - 5s 2ms/step - loss: 0.5109 - accuracy: 0.7586
Epoch 9/12
2814/2814 [==============================] - 5s 2ms/step - loss: 0.5081 - accuracy: 0.7585
Epoch 10/12
2814/2814 [==============================] - 5s 2ms/step - loss: 0.505